# Extract phenotype data from ICD10 for PheWAS analysis

## As-Is Software Disclaimer

This notebook is delivered "As-Is". Notwithstanding anything to the contrary, DNAnexus will have no warranty, support, liability or other obligations with respect to Materials provided hereunder.

[MIT License](https://github.com/dnanexus/UKB_RAP/blob/main/LICENSE) applies to this notebook.

## Jupyterlab app details (launch configuration)

Recommended configuration
- Runtime: ~20 min
- Cluster configuration: `Single Node`
- Recommended instance: `mem2_ssd1_v2_x32`
- Cost: ~£0.5

## Dependencies
|Library |License|
|:------------- |:-------------|
|[pandas](https://pandas.pydata.org/) |[BSD-3](https://github.com/pandas-dev/pandas/blob/main/LICENSE)|
|[numpy](https://numpy.org/) |[BSD-3](https://github.com/numpy/numpy/blob/main/LICENSE.txt)|
|[bgenix](https://enkre.net/cgi-bin/code/bgen/doc/trunk/doc/wiki/bgenix.md) | [Boost Software License (MIT-like)](https://enkre.net/cgi-bin/code/bgen/file?name=LICENSE_1_0.txt&ci=trunk)|
|[PLINK2](https://www.cog-genomics.org/plink/2.0/) |[GPL](https://github.com/chrchang/plink-ng/blob/master/2.0/COPYING)|


## Introduction
This notebook:
- Retrieves ICD10 codes for samples
- Create phenotype table in long format for subsequent PheWAS analysis
- Cleans covariates file
- Extracts genetic data, one file per variant
- Uploads data to UKB RAP

## Prepare your environment

Uncomment the install commands if you are comfortable with the library license and want to install and run the parts notebook that depend on the library.

In [3]:
%%capture captured
%%bash
# Install PLINK2
cd /opt/notebooks
wget https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220814.zip
unzip -o plink2_linux_avx2_20220814.zip

CalledProcessError: Command 'b'# Install PLINK2\ncd /opt/notebooks\nwget https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220814.zip\nunzip -o plink2_linux_avx2_20220814.zip\n'' returned non-zero exit status 9.

In [4]:
./plink2 --version

SyntaxError: invalid syntax (1996643621.py, line 1)

In [ ]:
%%capture captured
%%bash
# Install bgenix
cd /opt/notebooks
wget http://code.enkre.net/bgen/tarball/release/bgen.tgz
tar xvfz bgen.tgz > /dev/null
cd bgen.tgz/
./waf configure 
./waf 
./build/test/unit/test_bgen
./build/apps/bgenix -g example/example.16bits.bgen –list
cd /opt/notebooks

In [3]:
import dxpy
import numpy as np
import pandas as pd
import re
import shutil
import subprocess
import glob
import os
import ast

## Load dataset description and select entity containing phenotypic data

In [6]:
# Automatically discover dispensed dataset ID and load the dataset
dispensed_dataset = dxpy.find_one_data_object(
    typename="Dataset", name="app*.dataset", folder="/", name_mode="glob"
)
dispensed_dataset_id = dispensed_dataset["id"]

In [7]:
# Get project ID
project_id = dxpy.find_one_project()["id"]

In [8]:
dataset = (":").join([project_id, dispensed_dataset_id])

Using the `-ddd` parameter will extract 3 dictionary files associated with the dataset.

The 3 dictionary files that are returned include:
1. `entity_dictionary` that contains the different tables resources that are available. The table we’re most interested in tends to be the participant table that contains the information about each participant.
2. `data_dictionary` that contains the different field names that we might want to include in our dataset.
3. `coding_dictionary` that contains a lookup for the values for some of the field names.


*Note: The following cell can only be run once. Otherwise, you'll need to delete the existing data tables in order to re-run*


In [9]:
cmd = ["dx", "extract_dataset", dataset, "-ddd", "--delimiter", ","]
subprocess.check_call(cmd)

0

Specify fields ID to retrieve.

- `eid` - Participant's ID
- `41270` - [Diagnoses - ICD10](https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=41270)

## Retrieve data

In this section we:
1. Use the `data_dictionary.csv` table to filter the list of field names and select ones of interest (i.e. 'eid' and 'p41270').
2. Next we concatenate the field names with entity name (i.e. 'participant')
3. Finally, we use this list of field names joined with entity names (i.e. 'participant.eid') to query for a dataset with only those fields.

In [10]:
field_ids = ["41270"]

In [11]:
path = os.getcwd()

In [12]:
data_dict_csv = glob.glob(os.path.join(path, "*.data_dictionary.csv"))[0]
data_dict_df = pd.read_csv(data_dict_csv)
data_dict_df.head()

/tmp/ipykernel_76/118131557.py:2: DtypeWarning: Columns (4,7,8,9,10,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_dict_df = pd.read_csv(data_dict_csv)


,entity,name,type,primary_key_type,coding_name,concept,description,folder_path,is_multi_select,is_sparse_coding,linkout,longitudinal_axis_type,referenced_entity_field,relationship,title,units
0,participant,eid,string,global,NaN,NaN,NaN,Participant Information,NaN,NaN,NaN,NaN,NaN,NaN,Participant ID,NaN
1,participant,p3_i0,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 0,seconds
2,participant,p3_i1,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 1,seconds
3,participant,p3_i2,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 2,seconds
4,participant,p3_i3,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 3,seconds


In [13]:
# The UKB participant tables have the following naming convention for the fields (or columns): "p<field id>_i<instance id>_a<array id>"
# This function is used to grab all field names (e.g. "p<field_id>_iYYY_aZZZ") given a list of field IDs and return string to pass into
# extract_dataset
def fields_for_id(field_id):
    field_names = ["eid"]
    for _id in field_id:
        select_field_names = list(
            data_dict_df[
                data_dict_df.name.str.match(r"^p{}(_i\d+)?(_a\d+)?$".format(_id))
            ].name.values
        )
        field_names += select_field_names

    field_names = [f"participant.{f}" for f in field_names]
    return ",".join(field_names)

In [14]:
field_names = fields_for_id(field_ids)
field_names

'participant.eid,participant.p41270'

In [15]:
# Load dataset
# Note: This cell can only be run once. Otherwise, you'll need to delete the existing data tables in order to re-run
# Note: There is no space separating the different fields
cmd = [
    "dx",
    "extract_dataset",
    dataset,
    "--fields",
    field_names,
    "--delimiter",
    ",",
    "--output",
    "pheno_dictionary.csv",
]
subprocess.check_call(cmd)

0

In [16]:
pheno_dict_csv = "pheno_dictionary.csv"
pheno = pd.read_csv(pheno_dict_csv)
pheno.head()

,participant.eid,participant.p41270
0,1000128,"[""C61"",""F419"",""K219"",""N320"",""N359"",""N991"",""Y83..."
1,1000424,NaN
2,1000569,"[""E780"",""E785"",""E789"",""I10"",""I200"",""I209"",""I25..."
3,1000639,"[""D125"",""D128"",""I849"",""K621"",""K635"",""K641"",""L2..."
4,1000686,"[""C435"",""D179"",""E780"",""H401"",""H409"",""I10"",""I20..."


In [17]:
# Rename column headers
pheno = pheno.rename(columns=lambda x: re.sub("participant.", "", x))
pheno.head()

,eid,p41270
0,1000128,"[""C61"",""F419"",""K219"",""N320"",""N359"",""N991"",""Y83..."
1,1000424,NaN
2,1000569,"[""E780"",""E785"",""E789"",""I10"",""I200"",""I209"",""I25..."
3,1000639,"[""D125"",""D128"",""I849"",""K621"",""K635"",""K641"",""L2..."
4,1000686,"[""C435"",""D179"",""E780"",""H401"",""H409"",""I10"",""I20..."


In [18]:
pheno.shape

(502137, 2)

## Create phenotype table in PheWAS format

Each row is containing one phenotype (ICD 10 diagnosis) for one participant. Therefore, when participant has multiple diagnoses, participants eid will appear multiple times.

In [19]:
# Convert the values in the p41270 column from a string representation of a list to an actual list
# using ast.literal_eval()
pheno = pheno.set_index("eid").dropna()
pheno["p41270"] = pheno["p41270"].apply(lambda x: ast.literal_eval(x))
pheno.head()

,p41270
eid,
1000128,"[C61, F419, K219, N320, N359, N991, Y836, Z854]"
1000569,"[E780, E785, E789, I10, I200, I209, I251, I259..."
1000639,"[D125, D128, I849, K621, K635, K641, L290, Z871]"
1000686,"[C435, D179, E780, H401, H409, I10, I209, I214..."
1000695,"[H811, I10, I340, I451, I48, I489, I639, J338,..."


In [20]:
pheno_long_all = pheno.explode("p41270").reset_index()
pheno_long_all["count"] = 1
pheno_long_all["vocabulary_id"] = "ICD10"
pheno_long_all.rename({"eid": "id", "p41270": "code"}, axis=1, inplace=True)
pheno_long_all = pheno_long_all[["id", "vocabulary_id", "code", "count"]]
pheno_long_all.head()

,id,vocabulary_id,code,count
0,1000128,ICD10,C61,1
1,1000128,ICD10,F419,1
2,1000128,ICD10,K219,1
3,1000128,ICD10,N320,1
4,1000128,ICD10,N359,1


In [21]:
pheno_long_all.shape

(7015611, 4)

### Save phenotype table as CSV file and upload it

In [22]:
pheno_long_all.to_csv("pheno_icd10_long.csv", index=False)

In [24]:
!dx upload pheno_icd10_long.csv --path /phewas/

dxpy.exceptions.ResourceNotFound: The folder could not be found in project-GvQgb10JjKY3XGbpZbGVg0v3, code 404. Request Time=1746425874.373736, Request ID=1746425874375-205934


## Get covariate files

In [1]:
!dx download -f /Data/ischemia_df.phe

[===========================================================>] Completed 44,887,031 of 44,887,031 bytes (100%) /opt/notebooks/scripts/ischemia_df.phehe


In [4]:
pheno = pd.read_csv("ischemia_df.phe", sep='\t')
pheno.head()

,FID,IID,sex,age,bmi,ever_smoked,hdl_cholesterol,ldl_cholesterol,hypertension,ischemia_cc,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
0,1001251,1001251,0,51,24.4,1.0,1.452362,3.570649,0,1,-14.2457,3.96257,-1.018640,2.277420,-4.21549,-0.100448,1.524450,-1.246880,1.368960,2.570260
1,1001445,1001445,1,63,30.2,1.0,0.973000,2.930000,1,1,-13.4369,3.02070,-3.195770,0.428490,6.21226,-1.478700,1.282360,-1.606150,0.199501,-4.308460
2,1003040,1003040,0,57,31.7,1.0,1.291000,2.345000,1,1,-13.4688,4.63916,-2.339450,0.502712,-2.63828,0.662364,0.931805,0.987020,-5.069820,2.115390
3,1004201,1004201,0,55,36.8,1.0,1.465000,4.716000,0,1,-11.7176,3.71645,-0.737745,1.572150,-1.20120,-1.681440,1.290020,-0.223860,-5.693960,-1.878020
4,1004219,1004219,0,45,37.1,0.0,1.483000,3.084000,1,1,-12.1573,2.01404,-2.064440,1.280780,-1.68080,2.864480,2.142890,-0.380478,-4.603690,0.058457


In [5]:
pheno[['FID']].to_csv('eids_to_keep.txt', index=False, sep='\t', header=False)

In [6]:
!dx upload eids_to_keep.txt --path /Data/

[===========================================================>] Uploaded 2,655,968 of 2,655,968 bytes (100%) eids_to_keep.txt
ID                                file-J0G6BzjJjKYBzvP4Vx26Pf32
Class                             file
Project                           project-GvQgb10JjKY3XGbpZbGVg0v3
Folder                            /Data
Name                              eids_to_keep.txt
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Mon May  5 07:15:11 2025
Created by                        eila_zhong
 via the job                      job-J0G50z8JjKY6pPyz0G5P7Qqy
Last modified                     Mon May  5 07:15:12 2025
Media type                        
archivalState                     "live"
cloudAccount                      "cloudaccount-dnanexus"


In [7]:
covars = pheno.loc[:, [col for col in pheno.columns if col not in ['FID', 'ischemia_cc', 'hdl_cholesterol', 'ldl_cholesterol', 'hypertension']]]
covars.rename(columns={'IID': 'id'}, inplace=True)

In [8]:
covars.to_csv('covariates.txt', index=False, sep='\t')

In [9]:
!dx upload covariates.txt --path /Data/PheWAS/

dxpy.exceptions.ResourceNotFound: The folder could not be found in project-GvQgb10JjKY3XGbpZbGVg0v3, code 404. Request Time=1746429335.9734356, Request ID=1746429335975-590845


## Get genetic data

Download results of LD clumping

In [ ]:
!dx download -f /Data/LD_clump/plink_all_ld_clumped_ld_clumped.clumped

In [ ]:
ld_results = pd.read_csv('plink_all_ld_clumped_ld_clumped.clumped', delim_whitespace=True)
ld_results.sort_values(by='P').head()

In [ ]:
ld_results.shape

In [ ]:
%%bash
# Create symlink for imputed data
DIR='/mnt/project/Bulk/Imputation/Imputation*from*genotype*(GEL)'
ln -sf $DIR /opt/notebooks/imputed
DIR2=/mnt/project/Bulk-DRL/GEL_imputed_sample_files_fixed/
ln -sf $DIR2 /opt/notebooks/samples

For each varinat in LD table, extract genotypes and create PLINK raw file.

In [ ]:
if os.path.exists('phewas_geno_data'):
    shutil.rmtree('phewas_geno_data')

os.mkdir('phewas_geno_data')

for rsid in ld_results['SNP'].values:
    chromosome = ld_results[ld_results['SNP'] == rsid]['CHR'].values[0]
    #rsid = 'rs10455872'
    print(f'rsID: {rsid}, chromosome: {chromosome}')
    new_bgen_name = f'phewas_geno_data/{rsid}.bgen'
    plink_output_prefix = f'phewas_geno_data/plink_{rsid}'
    with open(new_bgen_name, 'wb') as new_bgen:
        print(f'Extract significant rsIDs')
        subprocess.check_call(['/opt/notebooks/bgen.tgz/build/apps/bgenix', '-g', f'imputed/ukb21008_c{chromosome}_b0_v1.bgen', 
                        '-incl-rsids', rsid], stdout=new_bgen, stderr=subprocess.PIPE)
        print(f'Make PLINK files')
        subprocess.check_call(['./plink2', '--bgen', new_bgen_name, 'ref-first', '--sample', 
                           f'samples/ukb21008_c{chromosome}_b0_v1.sample', '--rm-dup', 'force-first', '--out', plink_output_prefix,
                           '--keep-fam', 'eids_to_keep.txt', '--export',  'A'])

Open each raw PLINK file and rename header to be in PheWAS format.

In [ ]:
if os.path.exists('phewas_geno_rsid'):
    shutil.rmtree('phewas_geno_rsid')

os.mkdir('phewas_geno_rsid')

for file in glob.glob('phewas_geno_data/*.raw'):
    print(file.split('/')[1])
    dosage = pd.read_csv(file, sep='\t')
    dosage = dosage.iloc[:, [0,-1]]
    dosage.rename(columns={dosage.columns[0]: 'id', dosage.columns[1]: dosage.columns[1].split('_')[0]}, inplace = True)
    dosage.to_csv(f'phewas_geno_rsid/{dosage.columns[1]}.txt', index=False, sep='\t')

Upload data to UKB RAP

In [ ]:
!dx upload -r phewas_geno_rsid/ --path /Data/PheWAS/phewas_geno_rsid/

## Output files

- Phenome data in long format containing ICD 10 codes per participant (`pheno_icd10_long.csv`)
- Covariates file (`covariates.txt`)
- List of participants EIDs to include in the analysis (`eids_to_keep.txt`)
- Table for each index variant in LD clumping containing 2 columns: participant ID and rsID genotyping results (`<rsid>.raw`)